# ChestX-Ray data

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import models.training as tr
import models.reportnet as M
import data.preprocessing as pr
from torchvision import transforms

batch_size = 2
# Load data
uids = np.unique(pr.projections.index)

# Image preprocessing 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((2048, 2048), antialias=False)
])
train_data, train_loader, val_data, val_loader, test_data, test_loader = pr.create_dataloaders(uids, pr.IMAGES_PATH, batch_size=batch_size, transform=transform)

# for i, batch in enumerate(train_loader):
#     print(f"Images batch: {batch[0].shape}")
#     print(f"Reports batch: {batch[1].shape}")
#     break

# train_reports = [torch.tensor(train_data[i][1]).unsqueeze(0) for i in range(len(train_data))]
# train_reports = torch.cat(train_reports, dim=0)
# print(train_reports.shape)
# print(train_reports[0])
# pr.print_sequence(train_reports[0].numpy(), train_data.idx2word)

[nltk_data] Downloading package punkt to /home/mdelucasg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# ReportNet (our model from scratch)

## Training

In [3]:
# Define the parameters
d_model = 128 # how many features do we have for each token --> embeding dimension
nhead = 2
num_layers = 6
dropout = 0.1
max_len = 60

model = M.ReportNet(pr.tokenizer.vocab_size, d_model, max_len, dropout)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

tr.fit(model, optimizer, criterion, train_loader, val_loader, epochs=1)

Epoch 1


/home/mdelucasg/dev/CS577_DeepLearning_Project/models/reportnet.py:105: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(out)


## Testing

In [3]:
print(f"Test loss: {tr.validation_loop(model, criterion, test_loader)}")

Test loss: 5.0636599858601885


## Predict

In [5]:
from matplotlib import pyplot as plt
# Get a sample from the test set
bimg, breport = next(iter(test_loader))

# Get the prediction
result = tr.predict(model, bimg, breport, max_length=20, SOS_token=test_data.word2idx[pr.SOS_TOKEN], EOS_token=test_data.word2idx[pr.EOS_TOKEN])
pr.print_sequence(breport[0].numpy(), test_data.idx2word)
pr.print_sequence(result, train_data.idx2word)
print()

/home/mdelucasg/dev/CS577_DeepLearning_Project/models/reportnet.py:105: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(out)


<s> The lungs and pleural spaces show no acute abnormality . Heart size is mildly enlarged , pulmonary vascularity within normal limits . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
stable opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities opacities



In [27]:
pr.print_sequence(breport[0].numpy(), test_data.idx2word)

<s> 2 images . Calcified granuloma , right lung base . Heart size and pulmonary vascular engorgement appear within limits of normal . Mediastinal contour is unremarkable . No focal consolidation , pleural effusion , or pneumothorax identified . No convincing acute bony findings . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
